In [1]:
#from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
from nltk.corpus import stopwords
from  nltk import FreqDist
from mod_4_nlp_project.functions import *
import string
import spacy
import seaborn as sns

In [2]:
df = pd.read_csv('mod_4_nlp_project/cleaned_reviews_dataframe')

In [3]:
df.head()

,reviews,sentiment,cleaned_reviews
0,Working with one of the best Shakespeare sourc...,0,work with one of the good shakespeare source t...
1,"Well...tremors I, the original started off in ...",0,well tremor i the original start off in and i ...
2,Ouch! This one was a bit painful to sit throug...,0,ouch this one be a bit painful to sit through ...
3,"I've seen some crappy movies in my life, but t...",0,-PRON- have see some crappy movie in -PRON- li...
4,"""Carriers"" follows the exploits of two guys an...",0,carrier follow the exploit of two guy and tw...


In [6]:
# Load the en_core_web_sm model
nlp = spacy.load('en_core_web_md')

In [7]:
text = df['cleaned_reviews'][0]

#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

['work', 'with', 'one', 'of', 'the', 'good', 'shakespeare', 'source', 'this', 'film', 'manage', 'to', 'be', 'creditable', 'to', '-PRON-', 'source', 'whilst', 'still', 'appeal', 'to', 'a', 'wide', 'audience', 'branagh', 'steal', 'the', 'film', 'from', 'under', 'fishburne', 'nose', 'and', 'there', 's', 'a', 'talented', 'cast', 'on', 'good', 'form']


In [8]:
text = df['reviews'][0]
# create list of sentence tokens
doc = nlp(text)
sents_list = []
for sent in doc.sents:
    sents_list.append(sent.text)
print(sents_list)

["Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.  ", "Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form."]


### StopWords

In [4]:
stopwords_list = stopwords.words('english')+['pron', '-PRON-']
print(stopwords_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### feature engineering - show how many words are in a review, how many sentences, how many punctuation marks

In [ ]:
# Load the en_core_web_sm model
nlp = spacy.load('en_core_web_md')

In [5]:
# Tokenize each item in the review column 
word_tokens = [word_tokenize(review) for review in df['reviews']]


# Create an empty list to store the length of reviews
len_tokens = []

# Iterate over the word_tokens list and determine the length of each item
for i in range(len(word_tokens)):
     len_tokens.append(len(word_tokens[i]))

# Create a new feature for the lengh of each review
df['n_words'] = len_tokens 

In [ ]:
text = df['cleaned_reviews'][0]

#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

In [ ]:
text = df['reviews'][0]
# create list of sentence tokens
doc = nlp(text)
sents_list = []
for sent in doc.sents:
    sents_list.append(sent.text)
print(sents_list)

## Modeling

In [24]:
# choose X, y
predictors = df['cleaned_reviews']
target = df['sentiment']

### BOW

In [69]:
#TRAIN/TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(predictors, target, random_state=333)

In [70]:
BOW_vect = CountVectorizer(ngram_range=(1, 3), min_df = 10, stop_words=stopwords_list)

X_train=BOW_vect.fit_transform(X_train)
X_test=BOW_vect.transform(X_test)

In [71]:
X_train = pd.DataFrame(X_train.toarray(), columns=BOW_vect.get_feature_names())
X_test = pd.DataFrame(X_test.toarray(), columns=BOW_vect.get_feature_names())

In [72]:
X_test.shape

(6250, 33330)

In [73]:
pickle_out = open("BOW_vect.pickle","wb")
pickle.dump(BOW_vect, pickle_out)
pickle_out.close()

#### Logistic Regression

In [74]:
#INSTANTIATE LOGISTIC REGRESSION
logreg = LogisticRegression(C = .009, random_state = 333)
# can choose a regularization C = ? to penalize so it doesnt overfit

#fit to training set
logreg.fit(X_train, y_train)

#predict on test set
y_pred_log = logreg.predict(X_test)


# checking accuracy
print('Test Accuracy score: ', accuracy_score(y_pred_log,y_test))

# checking F1
print('Test F1 score: ', f1_score(y_pred_log,y_test))

# print confusion matrix
print('Confusion matrix test set: \n', confusion_matrix(y_test, y_pred_log)/len(y_test))

Test Accuracy score:  0.88032
Test F1 score:  0.8821304758903247
Confusion matrix test set: 
 [[0.43248 0.06624]
 [0.05344 0.44784]]


In [75]:
import pickle


pickle_out = open("log_reg_mod.pickle","wb")
pickle.dump(logreg, pickle_out)
pickle_out.close()

#### Naive Bayes

In [12]:
from sklearn.naive_bayes import MultinomialNB
# fit the training dataset on the NB classifier
Naive = MultinomialNB(alpha=0.1, fit_prior= True)
Naive.fit(X_train,y_train)
#prediction
nb_train_pre=Naive.predict(X_train)
nb_test_pre=Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Train Accuracy Score :",accuracy_score(nb_train_pre, y_train))
print("Naive Bayes Test Accuracy Score :",accuracy_score(nb_test_pre, y_test))
# calculate f1 scores for test data and train data 
nb_f1_score_test=f1_score(y_test,nb_test_pre)
nb_f1_score_train=f1_score(y_train,nb_train_pre)

print("Train data f1 score:{}".format(nb_f1_score_train))
print("Test data f1 score:{}".format(nb_f1_score_test ))
print("confusion_matrix:{}".format(confusion_matrix(y_test, nb_test_pre)))


Naive Bayes Train Accuracy Score : 0.9085333333333333
Naive Bayes Test Accuracy Score : 0.87104
Train data f1 score:0.908625925728595
Test data f1 score:0.8717377466581796
confusion_matrix:[[2705  412]
 [ 394 2739]]


#### Decision Tree

In [13]:
# INSATNTIATE DECISION TREE MODEL
tree = DecisionTreeClassifier(class_weight = 'balanced',random_state = 333)

tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)

# checking accuracy
print('Test Accuracy score: ', accuracy_score(y_test, y_pred_tree))

# checking F1
print('Test F1 score: ', f1_score(y_test, y_pred_tree))

# print confusion matrix
print('Confusion matrix test set: \n', confusion_matrix(y_test, y_pred_tree)/len(y_test))

Test Accuracy score:  0.72896
Test F1 score:  0.727652733118971
Confusion matrix test set: 
 [[0.36688 0.13184]
 [0.1392  0.36208]]


#### Random Forest

In [14]:
#INSTANTIATE RANDOM FOREST MODEL
rfc = RandomForestClassifier(random_state = 333, class_weight='balanced')

rfc.fit(X_train, y_train)

y_pred_forest = rfc.predict(X_test)

# checking accuracy
print('Test Accuracy score: ', accuracy_score(y_test, y_pred_forest))

# checking F1
print('Test F1 score: ', f1_score(y_test, y_pred_forest))
# print confusion matrix
print('Confusion matrix test set: \n', confusion_matrix(y_test, y_pred_forest)/len(y_test))

Test Accuracy score:  0.8576
Test F1 score:  0.8602825745682889
Confusion matrix test set: 
 [[0.4192  0.07952]
 [0.06288 0.4384 ]]


### TFIDF

In [62]:
#TRAIN/TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(predictors, target, random_state=333)

In [63]:
TFIDF_vect = TfidfVectorizer(ngram_range=(1, 3), min_df = 10, stop_words=stopwords_list)

X_train=TFIDF_vect.fit_transform(X_train)
X_test=TFIDF_vect.transform(X_test)

In [64]:
X_train = pd.DataFrame(X_train.toarray(), columns=TFIDF_vect.get_feature_names())
X_test = pd.DataFrame(X_test.toarray(), columns=TFIDF_vect.get_feature_names())

In [65]:
X_test.shape

(6250, 33330)

In [66]:
pickle_out = open("TFIDF_vect.pickle","wb")
pickle.dump(TFIDF_vect, pickle_out)
pickle_out.close()

#### Logistic Regression

In [19]:
#INSTANTIATE LOGISTIC REGRESSION
logreg = LogisticRegression(C = .009, random_state = 333)
# can choose a regularization C = ? to penalize so it doesnt overfit

#fit to training set
logreg.fit(X_train, y_train)

#predict on test set
y_pred_log = logreg.predict(X_test)


# checking accuracy
print('Test Accuracy score: ', accuracy_score(y_pred_log,y_test))

# checking F1
print('Test F1 score: ', f1_score(y_pred_log,y_test))

# print confusion matrix
print('Confusion matrix test set: \n', confusion_matrix(y_test, y_pred_log)/len(y_test))

Test Accuracy score:  0.83184
Test F1 score:  0.8396643783371471
Confusion matrix test set: 
 [[0.39152 0.1072 ]
 [0.06096 0.44032]]


#### Naive Bayes

In [67]:
from sklearn.naive_bayes import MultinomialNB
# fit the training dataset on the NB classifier
Naive = MultinomialNB(alpha=0.1, fit_prior= True)
Naive.fit(X_train,y_train)
#prediction
nb_train_pre=Naive.predict(X_train)
nb_test_pre=Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Train Accuracy Score :",accuracy_score(nb_train_pre, y_train))
print("Naive Bayes Test Accuracy Score :",accuracy_score(nb_test_pre, y_test))
# calculate f1 scores for test data and train data 
nb_f1_score_test=f1_score(y_test,nb_test_pre)
nb_f1_score_train=f1_score(y_train,nb_train_pre)

print("Train data f1 score:{}".format(nb_f1_score_train))
print("Test data f1 score:{}".format(nb_f1_score_test ))
print("confusion_matrix:{}".format(confusion_matrix(y_test, nb_test_pre)))


Naive Bayes Train Accuracy Score : 0.92592
Naive Bayes Test Accuracy Score : 0.87296
Train data f1 score:0.9265817432211004
Test data f1 score:0.8751572327044024
confusion_matrix:[[2673  444]
 [ 350 2783]]


In [68]:
pickle_out = open("nb_mod.pickle","wb")
pickle.dump(Naive, pickle_out)
pickle_out.close()

#### Decision Tree

In [21]:
# INSATNTIATE DECISION TREE MODEL
tree = DecisionTreeClassifier(class_weight = 'balanced',random_state = 333)

tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)

# checking accuracy
print('Test Accuracy score: ', accuracy_score(y_test, y_pred_tree))

# checking F1
print('Test F1 score: ', f1_score(y_test, y_pred_tree))

# print confusion matrix
print('Confusion matrix test set: \n', confusion_matrix(y_test, y_pred_tree)/len(y_test))

Test Accuracy score:  0.71968
Test F1 score:  0.7203957867858283
Confusion matrix test set: 
 [[0.35856 0.14016]
 [0.14016 0.36112]]


#### Random Forest

In [22]:
#INSTANTIATE RANDOM FOREST MODEL
rfc = RandomForestClassifier(random_state = 333, class_weight='balanced')

rfc.fit(X_train, y_train)

y_pred_forest = rfc.predict(X_test)

# checking accuracy
print('Test Accuracy score: ', accuracy_score(y_test, y_pred_forest))

# checking F1
print('Test F1 score: ', f1_score(y_test, y_pred_forest))
# print confusion matrix
print('Confusion matrix test set: \n', confusion_matrix(y_test, y_pred_forest)/len(y_test))

Test Accuracy score:  0.85552
Test F1 score:  0.8557277520370666
Confusion matrix test set: 
 [[0.42704 0.07168]
 [0.0728  0.42848]]
